# Amplitude Amplification Algorithm

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

## 1. Data Discretization

First we create the discretized probability $p(x)$ and function $f(x)$ needed for calculating expected value $E_{x\sim p}(f)$

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 10
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. Circuit Implementation

Next we execute **K** steps of the Amplitude Amplification operator $\hat{Q}$

In [ ]:
K=1
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)
qbits = Qprog.registers
for k in range(K):
    print(k)
    Qprog.apply(Q_Gate, qbits)

circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])

In [ ]:
%qatdisplay circuit

In [ ]:
result = run_job(lineal_qpu.submit(job))
Phi = postprocess_results(result)

In [ ]:
Phi

So we have:

* Quantum state before $\hat{Q}$: $|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$
* Quantum state after K applications of $\hat{Q}$: $\hat{Q}^{K}|\Psi\rangle = \sin{((2*K+1)\theta)}|\Psi_{1}\rangle+\cos{((2*K+1)\theta})|\Psi_{0}\rangle$

Now, we can measure the last qbit and generate the associate probabilities to $|\Psi_{1}\rangle$ y $|\Psi_{1}\rangle$:

* $P_{|\Psi_{1}\rangle}$: Probability of get state $|\Psi_{1}\rangle$ (we desire a high probability here)
* $P_{|\Psi_{0}\rangle}$: Probability of get state $|\Psi_{0}\rangle$ (we desire a low probability here)

Next we associate the probabilities with the amplitudes of $\hat{Q}^{K}|\Psi\rangle$:

$$\sin^{2}((2*K+1)\theta) = P_{|\Psi_{1}\rangle}$$
$$\sin((2*K+1)\theta) = \sqrt{P_{|\Psi_{1}\rangle}}$$
$$(2*K+1)\theta = \arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}$$

Now we have to stop because $\arcsin$ is a multivalued function but numpy only calculates the angle in the range: $[\frac{-\pi}{2}, \frac{\pi}{2}]$. If the rotation generated by $\hat{Q}^{K}$ is outside the range we cannot get the real rotated angle (ie $(2*K+1)\theta$)

In following cells we can see this behavior.

First we calculate:  $\theta = \frac{\arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}}{(2*K+1)}$

In [ ]:
#This is the angle asociated to the Phi state
theta_K = np.arcsin((Phi['Probability'].iloc[1])**0.5)
print('theta_K: {}'.format(theta_K))
thetaFromK = theta_K/(2*K+1)
print('thetaFromK: {}'.format(thetaFromK))

We know that:

$$\sin \theta_{th} = \sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}$$

$$ \theta_{th} = \arcsin{\sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}}$$

This is done in following cell

In [ ]:
theoric_theta = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))

print('thetaFromK == theoric_theta? {}'.format(np.isclose(thetaFromK, theoric_theta)))

But if we compute:

$$\sin^2((2*K+1)\theta$$

We get $P_{|\Psi_{1}\rangle}$

In [ ]:
print('|Psi_1> from  theoric_theta: {}'.format(np.sin((2*K+1)*theoric_theta)**2))
print('|Psi_1> from  Q^KPhi Probability: {}'.format(Phi['Probability'].iloc[1]))
print('Probability from Q^Phi and from theoric_theta are the same? {}'.format(
    np.isclose(np.sin((2*K+1)*theoric_theta)**2, Phi['Probability'].iloc[1])

))


So probabilities from $\hat{Q}^{K}|\Psi\rangle$ and from $\theta_{th}$ are the same but we cannot calculates the correct angle from the first because of multivalued $\arcsin$

## Quantum Amplification Amplitude

In [ ]:
from qat.lang.AQASM import H
Qprog, P_Gate, R_gate = ExpectationLoadingData(p_X, f_X)
qbits = Qprog.registers[0]
Q_Gate = Load_Q_Gate(P_Gate, R_gate)
n_aux = 10
qAux = Qprog.qalloc(n_aux)
for i, aux in enumerate(qAux):
    Qprog.apply(H, aux)
    for _ in range(2**i):
        Qprog.apply(Q_Gate.ctrl(), aux, qbits)
from qat.lang.AQASM.qftarith import QFT
Qprog.apply(QFT(n_aux).dag(), qAux)
circuit = Qprog.to_circ(submatrices_only=True)

In [ ]:
%qatdisplay circuit

In [ ]:
job = circuit.to_job(qubits=qAux)
result = RunJob(lineal_qpu.submit(job))
Phi = PostProcessResults(result)
Phi['Thetas'] = [m*np.pi/(2**n_aux) for m in list(Phi.index)] 

In [ ]:
%matplotlib inline
plt.plot(Phi['Thetas'], Phi['Probability'])

In [ ]:
theta_teorico = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theta_teorico: {}'.format(theta_teorico))
ThetaQ = Phi[Phi['Probability'] == max(Phi['Probability'])]['Thetas'].iloc[0]
print('theta cuantico: {}'.format(ThetaQ))
print('Integral Teorica: {}'.format(sum(p_X*f_X)))
print('Integral AA: {}'.format(np.sin(ThetaQ)**2))
print(u'Exact value of the integral:', IntegralExacta)

In [ ]:
Phi[Phi['Probability'] == max(Phi['Probability'])]['Thetas'].iloc[0]

In [ ]:
LowerLimit = 0.0
UpperLimit = 1.0 

In [ ]:
2.159845-0.981748

In [ ]:
Phi[Phi['Probability'] == max(Phi['Probability'])]

In [ ]:
10/(2**n_aux)*np.pi    

In [ ]:
9.5/(2**n_aux)*np.pi    

In [ ]:
theta_teorico

In [ ]:
max(Phi['Probability'])aa

In [ ]:
np.pi/2

In [ ]:
2*theta_teorico

In [ ]:
2*Phi.loc[10]['Thetas']

In [ ]:
2*Phi.loc[22]['Thetas']-2*np.pi

In [ ]:
np.sin(Phi.loc[22]['Thetas'])

In [ ]:
np.sin(Phi.loc[10]['Thetas'])

In [ ]:
np.sqrt(sum(p_X*f_X))